In [9]:
import pandas as pd


Read CSV files into Dataframe

In [10]:
df_bigdata = pd.read_csv("RawData/big_query-canada.csv")
df_province= pd.read_csv('RawData/Canada_province_coordinates.csv')

1. Add column category that is based on term, if term == 'lotto max' and "lotto max winning numbers" then "entertainment", else if term == "david suzuki" and "delonte west" then "people" else "sports"

In [11]:
df_bigdata['category'] = 'sports'
df_bigdata.loc[(df_bigdata['term'].isin(['lotto max','lotto max winning numbers'])), 'category'] = 'entertainment'
df_bigdata.loc[(df_bigdata['term'].isin(['david suzuki','delonte west'])), 'category'] = 'people'

Join Dataframes on common column 'Province' in df_province and 'Province' in df_bigdata

In [12]:
merged_df = pd.merge(df_bigdata, df_province, left_on= 'region_name', right_on='Province', how='inner')

Drop the redundant 'Province' column

In [13]:
merged_df.drop(columns=['Province'], inplace=True)

Perform further transformations as needed

# Perform further transformations
# Convert 'refresh_date' and 'week' columns to datetime objects

In [14]:
merged_df['refresh_date'] = pd.to_datetime(merged_df['refresh_date'])
merged_df['week'] = pd.to_datetime(merged_df['week'])

Rename columns if needed

In [15]:
merged_df.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'}, inplace=True)

the 'region_name' and 'country_name' columns will be converted to lowercase before further processing.

In [16]:
merged_df['region_name'] = merged_df['region_name'].str.lower()
merged_df['country_name'] = merged_df['country_name'].str.lower()

In [17]:
# Check for null values in 'longitude' and 'latitude' columns
null_coords = merged_df[merged_df['longitude'].isnull() | merged_df['latitude'].isnull()]
# Display the regions with null longitude and latitude
print("Regions with null longitude or latitude:")
print(null_coords[['region_name', 'country_name']])

Regions with null longitude or latitude:
Empty DataFrame
Columns: [region_name, country_name]
Index: []


Remove specific region names from 'region_name' column

In [18]:
region_to_remove = ['QuÃ©bec', 'Yukon Territory']
merged_df = merged_df[~merged_df['region_name'].isin(region_to_remove)]

In [19]:

merged_df.to_csv('cleaned_data.csv', index = False)

# Load Dataset into SQLlite

In [20]:
import sqlite3
# Create a SQLite database connection
conn = sqlite3.connect('ETL_project_db\etl_project.db')

In [21]:
# Create a cursor object
cursors = conn.cursor()

In [22]:
#create table in the db
create_table_query = '''
CREATE TABLE IF NOT EXISTS merged_data (
    country_name TEXT,
    region_name TEXT,
    refresh_date DATE,
    week DATE,
    rank INTEGER,
    score INTEGER,
    percent_gain INTEGER,
    term TEXT,
    category TEXT,
    latitude REAL,
    longitude REAL
);
'''
cursors.execute(create_table_query)

Insert data into the table

In [23]:
merged_df.to_sql('merged_data', conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Data successfully loaded into SQLite database.")

Data successfully loaded into SQLite database.


# SQL QUERY

In [4]:
import sqlite3
conn = sqlite3.connect('ETL_project_db\etl_project.db')
cursors = conn.cursor()
#Execute a sql query
cursors.execute("SELECT * FROM merged_data where category = 'people' and term = 'david suzuki'")

#fecth the rsults
results = cursors.fetchall()

for row in results:
    print(row)

('canada', 'alberta', '2023-04-10 00:00:00', '2023-04-02 00:00:00', 13, 6, 140, 'david suzuki', 'people', 'Canada', 53.01669802, -112.8166386)
('canada', 'ontario', '2023-04-10 00:00:00', '2023-04-02 00:00:00', 13, 27, 140, 'david suzuki', 'people', 'Canada', 44.56664532, -80.84998519)
('canada', 'british columbia', '2023-04-10 00:00:00', '2023-04-02 00:00:00', 13, 21, 140, 'david suzuki', 'people', 'Canada', 49.09996035, -116.516697)
('canada', 'alberta', '2023-04-10 00:00:00', '2023-03-26 00:00:00', 13, 4, 140, 'david suzuki', 'people', 'Canada', 53.01669802, -112.8166386)
('canada', 'british columbia', '2023-04-10 00:00:00', '2023-03-26 00:00:00', 13, 15, 140, 'david suzuki', 'people', 'Canada', 49.09996035, -116.516697)
('canada', 'manitoba', '2023-04-10 00:00:00', '2023-03-26 00:00:00', 13, 28, 140, 'david suzuki', 'people', 'Canada', 50.15002545, -96.88332178)
('canada', 'ontario', '2023-04-10 00:00:00', '2023-03-26 00:00:00', 13, 15, 140, 'david suzuki', 'people', 'Canada', 44.5

In [5]:
conn.close

<function Connection.close()>

# Finished